## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# tiny shakespeare 데이터셋 다운로드. 모델 학습에 사용
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-03 05:25:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-06-03 05:25:10 (101 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# 텍스트 데이터 읽기
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
# 데이터 셋의 총 문자 길이
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# 처음 1000개의 문자 출력
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



- 고유 문자 확인

In [6]:
# 텍스트에 포함된 모든 고유 문자를 정렬하여 리스트로 만듦
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars)) # 고유 문자들 출력
print(vocab_size) # 고유 문자 수


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


- 문자-정수 매핑 생성

In [8]:
# 문자를 정수로, 정수를 문자로 매핑하는 딕셔너리 생성
stoi = { ch:i for i,ch in enumerate(chars) }

itos = { i:ch for i,ch in enumerate(chars) }
# display(stoi, itos)
# 문자열을 정수 리스트로 인코딩하는 함수와 그 반대의 디코딩 함수를 정의
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


- 전체 텍스트 인코딩 및 텐서 저장

In [9]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
# 전체 텍스트를 인코딩해서 pytorch 텐서로 저장
data = torch.tensor(encode(text), dtype=torch.long)
# 텐서 형태, 데이터 타입 출력
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

- 데이터 train.val set으로 분리

In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] # 90% trainset data
val_data = data[n:] # 나머지 10% validation set

- 모델 입력 및 타겟 데이터 준비
 - block_size: 모델이 예측을 수행할 때 사용하는 컨텍스트 (or 입력 시퀀스)의 길이
 즉, 얼마나 많은 이전 문자를 보고 다음 문자를 예측할 지 결정
 - x: 특정 시점에서의 입력 시퀀스
 - y: x의 각 시퀀스에 대해 다음에 오는 문자가 타겟
-> 시퀀스 데이터에서 패턴 학습하는데 효과적

In [15]:
block_size = 8
print(train_data[:block_size])
print(train_data[1:block_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [ ]:
# 각 타임스텝에서의 컨텍스트와 타겟 출력
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


- 데이터 배치 생성 (Pytorch 사용)

In [17]:
torch.manual_seed(1337) # 시드 설정
batch_size = 4 # 병렬 처리할 독립 시퀀스의 수 정의
block_size = 8 # 예측에 사용될 최대 컨텍스트 길이

# train/val 에서 작은 배치로 나누기
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # 데이터 길이를 고려해서 랜덤한 시작 인덱스 선택 (선택된 시작 위치에서 block_size만큼 데이터 가져와도 인덱스 오류 안나게끔 하기 위해서)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(f'ix: {ix}')
    # 시작 인덱스로부터 block_size만큼 데이터 선택해서 입력 데이터 x를 생성
    x = torch.stack([data[i:i+block_size] for i in ix])
    # 동일한 시작 인덱스에서 한 칸 뒤로 이동한 위치부터 데이터를 선택하여 타겟 데이터 y를 생성
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [18]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


- 언어 모델 및 손실 함수 정의
 - B (Batch Size): 한 번에 처리되는 데이터 시퀀스의 수
 - T (Sequence Length): 각 입력 데이터(배치 내의 각 시퀀스)의 길이입니다. block_size에 해당하는 값, 모델이 한 번에 처리할 입력의 길이를 의미함
 - C (Number of Classes or Vocabulary Size): 모델 출력의 차원 수로, 가능한 각 토큰에 대한 점수를 의미. 어휘(vocabulary)의 크기와 같으며, 모델이 각 시퀀스 위치에서 예측할 수 있는 가능한 모든 토큰의 수.

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    # 빅램 언어 모델 정의
    def __init__(self, vocab_size):
        super().__init__()
        # 각 토큰에 대해 다음 토큰의 로짓을 직접 읽을 수 있는 룩업 테이블을 생성
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        # 해당 인덱스에 대한 임베딩 조회 B: 배치 크기, T: 시퀀스 길이, C: 클래스의 수
        logits = self.token_embedding_table(idx) # (B, T, C) 형태의 로짓 얻음

        if targets is None:
            loss = None # 타겟이 없다면 손실 없음
        else:
            print(f'logits.shape: {logits.shape}')
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # 텐서 변환
            print(f"target: {targets.shape}")
            targets = targets.view(B*T)
            print(f"target: {targets.shape}")
            # cross entropy 손실 함수 사용
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # 새 토큰 생성
        # idx: 현재 컨텍스트의 인덱스 배열 (B, T)
        for _ in range(max_new_tokens):
            # 현재 컨텍스트에 대한 예측 가져오기
            logits, loss = self(idx)
            # 마지막 단계 로짓만 가져오기
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities # 확률로 변환
            probs = F.softmax(logits, dim=-1) # (B, C) c: 전체 단어 수
            # sample from the distribution # # 확률 분포에서 샘플링하여 다음 토큰의 인덱스를 얻음
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # 샘플링된 인덱스를 현재 시퀀스에 추가
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) # 로짓의 형태 출력
print(loss) # 계산된 손실 출력
# 모델을 사용하여 새로운 토큰 생성 및 디코드된 결과 출력
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


logits.shape: torch.Size([32, 8, 65])
target: torch.Size([32, 8])
target: torch.Size([256])
torch.Size([256, 65])
tensor(4.7007, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


  -- cross entropy loss
: 모델의 예측 확률 분포와 실제 레이블의 확률 분포 사이의 차이 계산

- 옵티마이저 설정 및 학습 루프

In [27]:
# Adam 옵티마이저를 초기화
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [31]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # train 데이터셋에서 반복적으로 무작위로 배치를 샘플링하여 수행
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # 이전 그래디언트를 초기화하여 그래디언트가 누적되지 않도록 함
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    # 모델 가중치 업데이트
    optimizer.step()

print(loss.item())


4.700694561004639


- generate함수 이용해서 학습된 모델을 사용해 새로운 토큰 생성
 1. 초기 컨텐스트 설정: idx = torch.zeros((1, 1), dtype=torch.long) // 하나의 배치 (B=1)가 하나의 토큰 (T=1)을 가지고, 토큰의 인덱스 0 -> 실제 어휘에선 첫번째 토큰 의미함
 2. 토큰 생성: 입력된 idx 텐서를 시작으로 최대 500개의 새로운 토큰을 순차적으로 생성
 max_new_tokens 수만큼 반복하면서 다음 토큰을 예측, 이를 현재 시퀀스에 추가함
 4. 내부 동작
 - 로짓과 확률 계산: 각 반복에서 self(idx)를 통해 현재 시퀀스의 로짓을 계산, 마지막 토큰에 대한 로짓만 분리해서 softmax함수 적용 -> 다음 토큰의 확률 분포 얻기
 - 샘플링: 확률 분포에서 하나의 토큰을 무작위로 샘플링. (torch.multinomial).
 - 시퀀스 업데이트: 새 토큰의 인덱스를 현재 시퀀스 idx에 추가. 길이가 점차 늘어나는 시퀀스 생성
 5. 결과 반환 및 디코딩: tolist()로 리스트로 변환, decode함수 통해 인덱스 리스트를 실제 텍스트로 변환. 각 인덱스를 해당하는 토큰으로 변환

In [29]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


SpJFfx;LgRyJknOEti.?I&-gPlLyulId?XlaInQ'q,lT$
3Q&sGlvHQ?mqSq-eON
x?SP fUAfCAuCX:bOlgiRQWN:Mphaw
tRLKuYXEaAXxrcq-gCUzeh3w!AcyaylgYWjmJM?Uzw:inaY,:C&OECW:vmGGJAn3onAuMgia!ms$Vb q-gCOcPcUhOnxJGUGSPJWT:.?ujmJFoiNL&A'DxY,prZ?qdT;hoo'dHooXXlxf'WkHK&u3Q?rqUi.kz;?Yx?C&u3Qbfzxlyh'Vl:zyxjKXgC?
lv'QKFiBeviNxO'm!Upm$srm&TqViqiBD3HBP!juEOpmZJyF$Fwfy!PlvWPFC
&WDdP!Ko,px
x
tREOE;AJ.BeXkylOVD3KHp$e?nD,.SFbWWI'ubcL!q-tU;aXmJ&uGXHxJXI&Z!gHRpajj;l.
pTErIBjx;JKIgoCnLGXrJSP!AU-AcbczR?aytqQmBxZb:txqfSBj$I&
gXxy,j,SYg


## The mathematical trick in self-attention

- torch 사용, weighted aggregation 하기
 - torch.sum(a,1,keepdim=True)
  - torch.sum: 텐서의 합
  - a 행렬에서 각 행의 합을 계산
  - dim=1: 행 방향(열을 따라) 합 계산
  - keepdim=True: 원래의 차원을 유지한 채 결과 반환. 결과의 차원 유지

In [47]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
# 3x3 단위 삼각 행렬 생성: torch.tril: 모든 원소가 1인 하단 삼각 행렬 생성
a = torch.tril(torch.ones(3, 3))
display(a)
# 각 행의 합이 1이 되도록 행렬 a를 정규화 함. 각 행이 확률분포를 나타내도록 함
display(torch.sum(a, 1, keepdim=True) )
a = a / torch.sum(a, 1, keepdim=True) # (3x3) / (3x1)
# 무작위 행렬 생성: 0부터 9 사이의 정수로 구성된 3x2 크기의 무작위 행렬을 생성하고, 이를 실수형(float)로 변환
b = torch.randint(0,10,(3,2)).float()
c = a @ b # weighted agg.
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

tensor([[1.],
        [2.],
        [3.]])

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [48]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


- 시간 축을 따라 각 시점까지의 평균 값을 계산하는 것으로, 이는 시간에 따른 누적 평균을 구하는 방법

In [49]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C) # t까지의 이전 타임스텝의 값을 선택
        print(f"xprev: {xprev}")
        xbow[b,t] = torch.mean(xprev, 0) #  각 채널별로 평균을 계산
        print(f"xbow: {xbow}")


xprev: tensor([[ 0.1808, -0.0700]])
xbow: tensor([[[ 0.1808, -0.0700],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]])
xprev: tensor([[ 0.180

In [50]:
# version 2: using matrix multiply for a weighted aggregation

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)  # xbow와 xbow2 두 텐서가 거의 같은지 비교

False

- torch.allclose 예시 코드

In [51]:
import torch

# 두 텐서를 생성합니다.
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.tensor([1.0, 2.00001, 3.0])

# 두 텐서의 값이 거의 같은지 비교합니다.
result = torch.allclose(a, b)
print(result)  # True, 허용 오차 내에서 거의 같습니다.


True


- masked_fill: 주어진 텐서에서 마스크(mask)가 True인 위치를 특정 값으로 채우는 함수, 특정 요소를 마스킹하고 다른 값으로 대체
-  소프트맥스 결과가 하삼각 행렬 부분만을 고려하도록 함

In [52]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # tril == 0 조건을 사용하여 상삼각 행렬(하삼각 행렬이 아닌 부분)을 찾음
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [59]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
print("tril: ", tril.shape)
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print("wei:",wei.shape)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

tril:  torch.Size([8, 8])
wei: torch.Size([4, 8, 8])


torch.Size([4, 8, 16])

In [54]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [55]:
wei[0].shape

torch.Size([8, 8])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

**Notes:**

- **Attention은 커뮤니케이션 메커니즘입니다.** 이를 방향성 그래프의 노드로 볼 수 있으며, 각 노드는 서로를 바라보고 데이터를 기반으로 가중 합을 통해 정보를 집계합니다.
- **공간의 개념이 없습니다.** Attention은 단순히 벡터 집합에 대해 동작합니다. 그래서 토큰을 위치적으로 인코딩해야 합니다.
- **배치 차원의 각 예시는 완전히 독립적으로 처리되며 서로 "대화"하지 않습니다.**
- "인코더" attention 블록에서는 `tril`을 사용하여 마스킹하는 단일 라인을 삭제하여 모든 토큰이 서로 커뮤니케이션할 수 있도록 합니다. 여기 있는 블록은 삼각형 마스킹을 가지고 있어 "디코더" attention 블록이라 불리며, 주로 언어 모델링과 같은 자회귀 설정에 사용됩니다.
- **"self-attention"**은 쿼리와 키 및 값이 동일한 소스에서 생성된다는 것을 의미합니다. "cross-attention"에서는 쿼리는 여전히 `x`에서 생성되지만, 키와 값은 다른 외부 소스(예: 인코더 모듈)에서 옵니다.
- **"Scaled" attention**은 추가로 `wei`를 `1/sqrt(head_size)`로 나눕니다. 이렇게 하면 입력 Q와 K가 단위 분산을 가질 때 `wei`도 단위 분산을 가지며, 소프트맥스가 너무 포화되지 않고 분산된 상태를 유지하게 됩니다.

---

이 설명은 Attention 메커니즘의 기본 개념과 이를 구현하는 방법, 그리고 Self-Attention과 Cross-Attention의 차이점을 다룹니다. 또한, Scaled Attention의 필요성과 이를 구현하는 방법에 대해 설명합니다.

In [76]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # Query와 Key의 내적을 구한 후 head_size의 제곱근으로 나눔
# Query와 Key 간의 유사성을 나타낸 후 스케일링 함: 소프트맥스 함수의 입력값이 지나치게 크지 않도록 조정하여 계산의 안정성과 정확성을 높여줌

- k,q,wei 분산 계산
 - 모든 요소를 하나의 집합으로 간주하여 전체 분산을 계산

In [73]:
## 예시
import torch

# 4x8x2 텐서를 임의의 값으로 초기화합니다.
a = torch.randn(2, 3, 2)

# k 텐서의 분산을 계산합니다.
variance = a.var()
print(variance)


tensor(1.7791)


In [75]:
a

tensor([[[-1.4875, -0.2232],
         [ 0.3911, -1.0279],
         [-0.4114, -0.1948]],

        [[ 2.2010,  1.4921],
         [-0.2110,  1.0472],
         [-2.2158, -1.8080]]])

In [74]:
# 첫 번째 차원에 대한 분산을 계산합니다. 각 값에서 평균을 뺀 값의 제곱의 평균
variance_dim0 = a.var(dim=0)
print(variance_dim0)


tensor([[6.8026, 1.4712],
        [0.1813, 2.1529],
        [1.6280, 1.3013]])


In [77]:
k.var()

tensor(1.0004)

In [78]:
q.var()

tensor(1.0447)

In [79]:
wei.var()

tensor(0.9695)

In [80]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [81]:
# 입력 벡터를 8배 스케일링
# 입력 값들이 더 커지거나 차이가 더 커지면, 소프트맥스 결과는 더욱 뾰족해지는(peaky) 경향이 있음
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)
  # LayerNorm1d 클래스 정의

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    # 클래스 초기화 메서드
    # dim: 입력 벡터의 차원 수
    # eps: 작은 수로, 분산이 0이 되는 것을 방지하기 위해 추가됨
    self.eps = eps
    # 학습 가능한 파라미터 gamma와 beta 초기화 (각각 1과 0으로 설정)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # 객체를 함수처럼 호출할 때 실행되는 메서드 (forward pass 계산)
    # x: 입력 텐서
    # 배치의 평균 계산 (각 벡터의 차원별로)
    xmean = x.mean(1, keepdim=True) # batch mean
    # 배치의 분산 계산 (각 벡터의 차원별로)
    xvar = x.var(1, keepdim=True) # batch variance
    # 정규화: 평균을 빼고 분산으로 나눠서 유닛 분산으로 만듬
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    # 정규화된 값에 gamma와 beta를 적용하여 출력 계산
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    # 학습 가능한 파라미터 (gamma와 beta) 반환
    return [self.gamma, self.beta]

# 난수 시드를 설정하여 재현 가능한 결과를 만듬
torch.manual_seed(1337)
# LayerNorm1d 모듈을 차원 수 100으로 초기화
module = LayerNorm1d(100)
# 크기가 (32, 100)인 무작위 텐서 생성 (배치 크기 32, 차원 100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
# 모듈을 호출하여 입력 텐서 x에 대해 정규화 수행
x = module(x)
# 결과 텐서의 크기 출력
x.shape


torch.Size([32, 100])

In [ ]:
# x의 첫번째 feature에 대한 평균과 표준편차 계산
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
# 첫 번째 샘플의 모든 특성 값에 대한 평균과 표준편차를 계산
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [83]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
context

tensor([[0]])

In [82]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5059
step 300: train loss 2.4196, val loss 2.4338
step 400: train loss 2.3504, val loss 2.3566
step 500: train loss 2.2965, val loss 2.3129
step 600: train loss 2.2410, val loss 2.2500
step 700: train loss 2.2057, val loss 2.2191
step 800: train loss 2.1633, val loss 2.1864
step 900: train loss 2.1244, val loss 2.1510
step 1000: train loss 2.1038, val loss 2.1308
step 1100: train loss 2.0707, val loss 2.1197
step 1200: train loss 2.0377, val loss 2.0800
step 1300: train loss 2.0268, val loss 2.0650
step 1400: train loss 1.9918, val loss 2.0356
step 1500: train loss 1.9697, val loss 2.0293
step 1600: train loss 1.9645, val loss 2.0499
step 1700: train loss 1.9404, val loss 2.0129
step 1800: train loss 1.9095, val loss 1.9951
step 1900: train loss 1.9067, val loss 1.9855
step 2000: train loss 1.8854, val loss 1.9948
step 2100: train loss 1.